In [1]:
from asteroid.models.fasnet import FasNetTAC
from IPython.display import display, Audio
import soundfile as sf
import torch
import json

In [2]:
# 'from_pretrained' automatically uses the right model class (asteroid.models.TasNetTAC).
model_path = "/Users/ccui/Desktop/asteroid/egs/AMIx/TAC/exp/snr_data_clips_synthesis_4array1_align_a_bs4_epo100_cf41b7ef/best_model.pth"
model = FasNetTAC.from_pretrained(model_path)

In [6]:
mix_path_list = ["/Users/ccui/Desktop/AMI_sample/clips_synthesis_8array1_align/sample391/mixture_mic1.wav","/Users/ccui/Desktop/AMI_sample/clips_synthesis_8array1_align/sample391/mixture_mic2.wav","/Users/ccui/Desktop/AMI_sample/clips_synthesis_8array1_align/sample391/mixture_mic3.wav","/Users/ccui/Desktop/AMI_sample/clips_synthesis_8array1_align/sample391/mixture_mic4.wav"]
display(Audio(mix_path_list[0]))

In [7]:
mixtures=[]
for mix_path in mix_path_list:
    mix, fs = sf.read(mix_path, dtype="float32") 
    mix = torch.from_numpy(mix).unsqueeze(0)
    mixtures.append(mix)
mixtures = torch.cat(mixtures, 0)
est_sources = model.separate(mixtures[None])

est_sources_np = est_sources.squeeze(0).cpu().data.numpy()
for src_idx, est_src in enumerate(est_sources_np):
    display(Audio(est_src,rate = 16000))
    sf.write("s{}_estimate.wav".format(src_idx + 1),est_src,16000)

In [3]:
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement

In [5]:
# Model is downloaded from the speechbrain HuggingFace repo
tmpdir = "results"
enhancer = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/mtl-mimic-voicebank",
    savedir=tmpdir,
)
# noisy, fs = torchaudio.load("/Users/ccui/Desktop/AMI_sample/clips_synthesis_8array1_align/sample391/mixture_mic4.wav")
# Channel dimension is interpreted as batch dimension here
# enhanced = enhancer.enhance_batch(noisy)

In [ ]:
for src_idx, est_src in enumerate(est_sources_np):
    est_src, fs = torchaudio.load("s{}_estimate.wav".format(src_idx + 1))
    enhanced = enhancer.enhance_batch(est_src)
    display(Audio(enhanced,rate = 16000))